In [1]:
import time
import numpy as np
import itertools
import math
import copy
from functools import reduce # python3 compatibility
from operator import mul
import json

In [2]:
# Functions for generating environment with capability
# generating task
def gen_tasks(task_num, max_capNum, capabilities): # n is the number of task, max_capNum is the maximum number of cap a task could require
    return [sorted(np.random.choice(capabilities, np.random.randint(1, max_capNum+1),replace=False)) for j in range(0, task_num)]


def gen_constraints(agent_num, task_num, power=1, a_min_edge=2,
                    t_max_edge=5):  # power is the inforce you put in the probabilities

    # the maximum tasks an agent could work on depends on the number of tasks available (e.g, if |T| = 1/2|A|, the roughly each agent can work on two tasks)
    # calculate the max and min edges for agents
    seats = math.floor(t_max_edge * task_num)
    a_taskInds = [[] for i in range(0, agent_num)]
    t_counter = [0 for j in range(0, task_num)]  # each indicate the current number of agents on the task

    ## generating the number of tasks the agents could work on.
    a_taskNums = []
    for i in range(0, agent_num):
        a_max_edge = min((seats - (agent_num - 1 - i) * a_min_edge), t_max_edge)
        a_min_edge = min(a_min_edge, a_max_edge)
        a_taskNums.append(
            np.random.randint(a_min_edge, a_max_edge + 1))  # indicate the number of task the agent could work on
        seats -= a_taskNums[i]

    t_indexes = [j for j in range(0, task_num) if
                 t_counter[j] < t_max_edge]  # make sure no further draw for those reached the maximum limit.
    for i in range(0, agent_num):
        if any([tc == 0 for tc in t_counter]):
            t_prob = [(math.e ** (t_max_edge - t_counter[j])) ** power for j in
                      t_indexes]  # power is used to manify the probability
            sum_prob = sum(t_prob)
            t_prop_2 = [prop / sum_prob for prop in t_prob]

            # draw tasks accounting to their current allocations
            a_taskInds[i] = list(np.random.choice(t_indexes, min(a_taskNums[i], len(t_indexes)), replace=False,
                                                  p=[prop / sum_prob for prop in t_prob]))
            # increase the chosen task counters
        else:
            a_taskInds[i] = list(np.random.choice(t_indexes, min(a_taskNums[i], len(t_indexes)), replace=False))

        for j in a_taskInds[i]:
            t_counter[j] += 1
        t_indexes = [j for j in range(0, task_num) if
                     t_counter[j] < t_max_edge]  # make sure no further draw for those reached the maximum limit.

    # get also the list of agents for each task
    t_agents = [[i for i in range(0, agent_num) if j in a_taskInds[i]] for j in range(0, task_num)]

    return a_taskInds, t_agents


def gen_agents(constraints, tasks, max_capNum, capabilities,
               max_capVal):  # m is the number of task, max_capNum is the maximum number of cap a task could require, max_capVal is the maximum capability value

    a_taskInds = constraints[0]
    agent_num = len(a_taskInds)
    caps_lists = []
    contri_lists = []
    for i in range(0, agent_num):
        t_caps = [tasks[j] for j in a_taskInds[i]]  # lists of caps that each task agent could perform

        caps_union = set(itertools.chain(*t_caps))  # union of caps of tasks that agent could perform
        a_cap_num = np.random.randint(min(3, max_capNum, len(caps_union)),
                                      min(len(caps_union), max_capNum) + 1)  # the num of caps the agent will have

        a_caps = set([np.random.choice(t_c) for t_c in
                      t_caps])  # initial draw to guarantee the agent has some contribution to each of the task that he could do

        rest_choices = list(caps_union.difference(a_caps))
        if rest_choices != []:
            # if min(len(rest_choices), a_cap_num - len(a_taskInds[i])) < 0:
            #     print("negative number a_cap_num - len(a_taskInds[i]): {}!".format(a_cap_num - len(a_taskInds[i])))
            # else:
            update_len = max(0, a_cap_num - len(a_taskInds[i]))
            a_caps.update(np.random.choice(rest_choices, min(len(rest_choices), update_len), replace=False))

        caps_lists.append(sorted(list(a_caps)))

        contri_lists.append(
            [(np.random.randint(1, max_capVal + 1) if c in caps_lists[i] else 0) for c in range(0, len(capabilities))])
    return caps_lists, contri_lists



In [3]:
def random(agents, tasks, constraints, gamma = 1):
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = constraints[0]
    alloc = [np.random.choice(a_taskInds[i]+[task_num]) for i in range(0,agent_num)]
    return alloc, sys_reward_agents(agents,tasks, alloc, gamma)

In [4]:
def task_reward(task, agents, gamma=1):  # task is represented by a list of capabilities it requires, agents is
    # a list of agents, where each represented by a list cap contribution values
    if agents == []:
        return 0
    else:
        return sum([max([agent[cap] for agent in agents]) for cap in task]) * (gamma ** len(agents))


def sys_reward_agents(agents, tasks, alloc, gamma=1): #alloc is a vector of size M each element indicate which task the agent is alloated to
    return sum([task_reward(tasks[j], [agents[i] for i in range(0,len(agents)) if alloc[i] == j], gamma) for j in range(0, len(tasks))])


def resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma=1):
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)
    a_msg_sum = [{d_key: sum([r_msgs[j][i][0] for j in a_taskInds[i] if j != d_key])
                         + r_msgs[d_key][i][1] for d_key in a_taskInds[i]}
                 for i in range(0, agent_num)]
    alloc = [max(ams, key=ams.get) if ams != {} else task_num for ams in a_msg_sum]
    return alloc, sys_reward_agents(agents, tasks, alloc, gamma), iteration, iter_over, converge


# Agent contribution
def agent_con(competency_lists, t_caps_lists, query_agentIndex, query_taskIndex, member, a_taskInds):
    # the marginal contribution of an agent to a coalition
    if query_taskIndex == len(t_caps_lists):
        return 0
    if query_taskIndex not in a_taskInds[query_agentIndex]:
        return 0
    curr_pro = [competency_lists[mem] for mem in member]
    cur_reward = task_reward(t_caps_lists[query_taskIndex], curr_pro, gamma=1)
    if query_agentIndex in member:
        remained_mems = member[:]
        remained_mems.remove(query_agentIndex)  # the remained coalition members besides of agent i
        coal_pro = [competency_lists[mem] for mem in remained_mems]
        new_reward = task_reward(t_caps_lists[query_taskIndex], coal_pro, gamma=1)
        return cur_reward - new_reward
    else:
        added_mems = member[:]
        added_mems.append(query_agentIndex)
        coal_pro = [competency_lists[mem] for mem in added_mems]
        new_reward = task_reward(t_caps_lists[query_taskIndex], coal_pro, gamma=1)
        return new_reward - cur_reward


def aim_tasks(movement_values, a_tasks, a_index):
    max_mv = max(movement_values['agent ' + str(a_index)])
    target_tasks = []  # aiming tasks are the tasks the agent makes proposal to
    max_num = movement_values['agent ' + str(a_index)].count(max_mv)
    first_pos = 0
    for ind in range(max_num):
        new_list = movement_values['agent ' + str(a_index)][first_pos:]
        index_add_t = first_pos + new_list.index(max_mv)
        act_task = a_tasks[a_index][index_add_t]
        target_tasks.append(act_task)
        next_pos = new_list.index(max_mv) + 1
        first_pos += next_pos
    return target_tasks


def accepted_tasks(instruction_received, propose_states, a_index):
    accept_num = instruction_received['agent ' + str(a_index)].count(1)
    accept_tasks = []  # the index of accept message, which corresponding to the index of sending tasks
    first_pos = 0
    for ind in range(accept_num):
        new_list = instruction_received['agent ' + str(a_index)][first_pos:]
        new_t_ind = first_pos + new_list.index(1)
        act_task = propose_states['agent ' + str(a_index)][new_t_ind]
        accept_tasks.append(act_task)
        next_pos = new_list.index(1) + 1
        first_pos += next_pos
    return accept_tasks



In [5]:
def OPD(agents, tasks, constraints, gamma):
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = [list(con) for con in constraints[0]]
    t_agentInds = [list(con) for con in constraints[1]]

    a_ubs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]
    a_lbs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]

    for j in range(0, task_num):

        linked_agentInds = t_agentInds[j]
        com_dict = []
        com_rewards = []
        for c in itertools.product(*[[0, 1] for i in linked_agentInds]):
            com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
            com_rewards.append(
                task_reward(tasks[j], [agents[a_key] for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1],
                            gamma)
            )

        for i in t_agentInds[j]:
            t_ind = a_taskInds[i].index(j)
            cons_j = [com_rewards[c]
                      for c in range(0, len(com_dict)) if com_dict[c][i] == 1]

            a_lbs[i][t_ind] = min(cons_j)
            a_ubs[i][t_ind] = max(cons_j)

    for i in range(0, agent_num):
        t_flag = [True for j in a_taskInds[i]]
        for t_ind in range(0, len(a_taskInds[i])):
            for t2_ind in range(0, len(a_taskInds[i])):
                if t_ind != t2_ind and a_ubs[i][t_ind] < a_lbs[i][t2_ind]:
                    t_flag[t_ind] = False
                    break

        for t_ind in range(0, len(t_flag)):
            if not t_flag[t_ind]:
                t_agentInds[a_taskInds[i][t_ind]].remove(i)

        new_a_taskInds = [a_taskInds[i][t_ind]
                          for t_ind in range(0, len(a_taskInds[i])) if t_flag[t_ind]]
        a_taskInds[i] = new_a_taskInds

    return a_taskInds, t_agentInds


def FMS(agents, tasks, constraints, gamma, time_bound):
    converge = False
    iter_over = False
    start_time = time.time()
    a_taskInds = constraints[0]
    t_agentInds = constraints[1]
    task_num = len(tasks)
    agent_num = len(agents)

    q_msgs = [{t_key: {} for t_key in a_taskInds[i]} for i in range(0, agent_num)]
    r_msgs = [
        {t_agentInds[j][i]: ({1: -100} if len(a_taskInds[t_agentInds[j][i]]) == 1 else {key: -100 for key in [0, 1]})
         for i in range(0, len(t_agentInds[j]))}
        for j in range(0, task_num)]

    q_flags = [False for i in range(0, agent_num)]
    r_flags = [False for j in range(0, task_num)]

    iteration = 0
    while iteration < agent_num + task_num:  # True:
        if time.time() - start_time >= time_bound:
            return resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
        iteration += 1

        if iteration > agent_num + task_num:
            iter_over = True
            return resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)

        if all(q_flags) and all(r_flags):  # converge, msgs are all the same.
            converge = True
            # break
        for i in range(0, agent_num):
            linked_taskInds = a_taskInds[i]

            flag = True
            for t_key in linked_taskInds:

                ####### check time bound
                if time.time() - start_time >= time_bound:
                    return resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound
                msgs = {}

                if len(linked_taskInds) > 1:
                    msgs[1] = sum([m[0] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
                    msg_0 = []
                    ts = list(linked_taskInds)
                    ts.remove(t_key)

                    for k in ts:
                        msg_0.append(sum([m[0] for m in [r_msgs[j][i] for j in ts if j != k]])
                                     + r_msgs[k][i][1])

                    msgs[0] = (0 if msg_0 == [] else max(msg_0))
                else:
                    msgs[1] = 0

                alphas = -sum(msgs.values()) / len(msgs.keys())

                msgs_regularised = {d_key: msgs[d_key] + alphas for d_key in msgs.keys()}

                old_msg = q_msgs[i][t_key]
                if old_msg != {} and any([abs(msgs_regularised[d_key] - old_msg[d_key]) > 10 ** (-5)
                                          for d_key in old_msg.keys()]):
                    flag = False

                q_msgs[i][t_key] = msgs_regularised

            if flag:  # agent i sending the same info
                q_flags[i] = True

        if time.time() - start_time >= time_bound:
            break
        ###################### SAME thing, using comprehension
        #             msgs = {t_key:{d_key:sum([m[d_key] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        #             alphas = {t_key:-sum(msgs[t_key].values())/len(msgs.keys())
        #                       for t_key in linked_taskInds}
        #             msgs_regularised = {t_key:{d_key:msgs[t_key][d_key] + alphas[t_key]
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        for j in range(0, task_num):
            linked_agentInds = t_agentInds[j]
            # msg_con = [q_msgs[a][j] for a in linked_agentInds]

            com_dict = []
            com_rewards = []
            dom_com = [[0, 1] if len(a_taskInds[i]) > 1 else [1] for i in linked_agentInds]

            for c in itertools.product(*dom_com):
                ####### check time bound
                if time.time() - start_time >= time_bound:
                    return resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound

                com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
                com_rewards.append(
                    task_reward(tasks[j], [agents[a_key] for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1],
                                gamma)
                )

            flag = True
            for a_key in linked_agentInds:

                ####### check time bound
                if time.time() - start_time >= time_bound:
                    return resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound

                old_msg = r_msgs[j][a_key]
                q_table = []
                for c in range(0, len(com_dict)):
                    q_table.append(sum([q_msgs[a][j][com_dict[c][a]] for a in linked_agentInds if a != a_key])
                                   + com_rewards[c])

                r_msgs[j][a_key] = {
                    d_key: max([q_table[c] for c in range(0, len(com_dict)) if com_dict[c][a_key] == d_key])
                    for d_key in ([0, 1] if len(a_taskInds[a_key]) > 1 else [1])}

                if any([abs(r_msgs[j][a_key][d_key] - old_msg[d_key]) > 10 ** (-5) for d_key in old_msg.keys()]):
                    flag = False

            if flag:  # task j sending the same info
                r_flags[j] = True

    return resultCal(agents, tasks, constraints, r_msgs, iteration, iter_over, converge, gamma)


In [6]:
def embed_DSA(a_tasks, t_agents, tasks, agents, probability, time_bound):
    converge = False
    agent_num = len(agents)
    task_num = len(tasks)

    received_contribution = {} # zero at the task_num-th is the contribution of an agent assigned to no tasks
    for i in range(0, agent_num):
        received_contribution['agent ' + str(i)] = list(np.zeros(len(a_tasks[i]), int))
    movement_values = {}
    CS = [[] for j in range(0, task_num + 1)]
    CS[task_num] = list(range(0, agent_num))
    state = [task_num for i in range(0, agent_num)]
    update_t = range(0, task_num)
    iter = 0
    message_pass_num = 0
    is_continue = True
    start_time = time.time()
    while is_continue:  # > 0:
        if time.time() - start_time >= time_bound:
            break
        iter += 1

        # update agents' allocation state
        for j in range(0, task_num):
            member_agents = CS[j]
            for i in member_agents:
                state[i] = int(j)

        # tasks calculate agents contribution to them
        for j in range(0, task_num):
            if j in update_t:
                for i in t_agents[j]:
                    message_pass_num += 1
                    j_index = a_tasks[i].index(j)
                    received_contribution['agent ' + str(i)][j_index] = agent_con(agents, tasks, i, j, CS[j], a_tasks)

        # agents calculate their movement values #
        for i in range(0, agent_num):
            current_task = state[i]   # the agent's current task
            if current_task == task_num:
                current_contribution = 0
            else:
                cur_t_ind = a_tasks[i].index(current_task)
                current_contribution = received_contribution['agent ' + str(i)][cur_t_ind]
            movement_values['agent ' + str(i)] = [received_contribution['agent ' + str(i)][mess_ind]-current_contribution for mess_ind in range(0, len(received_contribution['agent ' + str(i)]))]

        # agents choose their tasks  ##########
        update_t = []
        max_mv = [[] for i in range(0, agent_num)]
        for i in range(0, agent_num):
            max_mv[i] = max(movement_values['agent ' + str(i)])
            if max_mv[i] > 0:
                pro = np.random.random(1)
                if pro > 1 - probability:
                    old_task = state[i]
                    potential_task = aim_tasks(movement_values, a_tasks, i)
                    choosed_task = np.random.choice(potential_task)
                    CS[choosed_task].append(i)
                    update_t.append(choosed_task)
                    state[i] = choosed_task
                    CS[old_task].remove(i)
                    update_t.append(old_task)
        #  if continue
        if max(max_mv) > 0:
            is_continue = True
        else:
            is_continue = False
            converge = True

    #  current system reward    #
    global_u = sys_reward_agents(agents, tasks, state, gamma=1)

    return global_u, iter, message_pass_num, converge


In [7]:
def embed_disNE(a_tasks, t_agents, tasks, agents, time_bound, CS=[]):
    converge = False
    agent_num = len(agents)
    task_num = len(tasks)
    if CS== []:  # start from scratch
        CS = [[] for j in range(0, task_num + 1)]  # current coalition structure, the last one is dummy coalition
        CS[task_num] = list(range(0, agent_num))
    state = [task_num for i in range(0, agent_num)]  # agents' current assignments, initially are no-task (task_num)
    
    received_contribution = {}  # zero at the task_num-th is the contribution of an agent assigned to no tasks
    for i in range(0, agent_num):
        received_contribution['agent ' + str(i)] = list(np.zeros(len(a_tasks[i]), int))

    movement_values = {}

    
    iter = 0
    update_t = range(0, task_num)
    message_pass_num = 0
    is_continue = True
    start_time = time.time()
    while is_continue:  # > 0:
        if time.time() - start_time >= time_bound:
            break
        iter += 1
        # update agents' allocation state
        for j in update_t:
            member_agents = CS[j]
            for i in member_agents:
                state[i] = int(j)

        # tasks calculate agents contribution to them
        for j in range(0, task_num):
            if j in update_t:
                for i in t_agents[j]:
                    message_pass_num += 1
                    j_index = a_tasks[i].index(j)
                    received_contribution['agent ' + str(i)][j_index] = agent_con(agents, tasks, i, j, CS[j], a_tasks)

        # agents calculate their movement values #
        for i in range(0, agent_num):
            current_task = state[i]   # the agent's current task
            if current_task == task_num:
                current_contribution = 0
            else:
                cur_t_ind = a_tasks[i].index(current_task)
                current_contribution = received_contribution['agent ' + str(i)][cur_t_ind]
            movement_values['agent ' + str(i)] = [received_contribution['agent ' + str(i)][mess_ind]-current_contribution for mess_ind in range(0, len(received_contribution['agent ' + str(i)]))]

        #   agents propose to tasks #
        received_proposal = [{'from agents': [], 'proposal values': []} for j in range(0, task_num)]   # the proposals received by tasks
        propose_states = {}    # which agents propose to which tasks
        propose_agents = []     # which agents make proposals
        received_tasks = []  # which tasks receive proposals
        max_mv = [[] for i in range(0, agent_num)]
        for i in range(0, agent_num):
            max_mv[i] = max(movement_values['agent ' + str(i)])
            if max(movement_values['agent ' + str(i)]) > 0:
                propose_agents.append(i)
                aim_task = aim_tasks(movement_values, a_tasks, i)
                old_task = state[i]
                if old_task != task_num:
                    aim_task.append(old_task)
                message_pass_num += len(aim_task)
                propose_states['agent ' + str(i)] = aim_task
                received_tasks.append(aim_task)
                for j in aim_task:
                    received_proposal[j]['from agents'].append(i)
                    received_proposal[j]['proposal values'].append(max_mv[i])

        # check whether or not to continue  #
        if max(max_mv) > 0:
            is_continue = True
        else:
            is_continue = False
            converge = True

        # tasks send instruction messages to agents #
        if is_continue:
            #  find out which coalitions received proposal
            active_tasks = []
            for j in received_tasks:
                active_tasks.extend(j)
            active_tasks = list(set(active_tasks))

            instruction_received = {}
            for i in propose_agents:
                instruction_received['agent ' + str(i)] = list(np.zeros(len(propose_states['agent ' + str(i)]), int))
                message_pass_num += len(propose_states['agent ' + str(i)])

            for j in active_tasks:
                if j == task_num:
                    for a_i in received_proposal[j]['from agents']:
                        a_i_index = propose_states['agent ' + str(int(a_i))].index(j)
                        instruction_received['agent ' + str(int(a_i))][a_i_index] = 1
                else:
                    max_pro = max(received_proposal[j]['proposal values'])
                    index_add_a = received_proposal[j]['proposal values'].index(max_pro)
                    choosed_agent = received_proposal[j]['from agents'][index_add_a]
                    acc_t_ind = propose_states['agent '+ str(int(choosed_agent))].index(j)   # the index of task j
                    instruction_received['agent '+str(int(choosed_agent))][acc_t_ind] = 1

        # agents move to tasks
        update_t = []
        for i in propose_agents:
            # accept_num = instruction_received['agent '+str(i)].count(1)
            if instruction_received['agent '+str(i)].count(1) == 0:
                continue
            else:
                accept_tasks = accepted_tasks(instruction_received, propose_states, i)
                old_coa = state[i]
                if old_coa != task_num:
                    old_coa_ind = propose_states['agent '+ str(i)].index(old_coa)
                    if instruction_received['agent '+str(i)][old_coa_ind] == 0:  # agent's own task reject its movement
                        continue
                    else:
                        message_pass_num += 1
                        accept_tasks.remove(old_coa)  # the other accept-coalitions besides its own coalition
                if len(accept_tasks) == 0:
                    continue
                else:
                    message_pass_num += 1
                    choosed_task = np.random.choice(accept_tasks)  # random select a coalition that gives accept message
                CS[choosed_task].append(i)
                update_t.append(choosed_task)
                de_i_ind = CS[old_coa].index(i)
                del CS[old_coa][de_i_ind]
                update_t.append(old_coa)

    # current system reward #
    global_u = sys_reward_agents(agents, tasks, state, gamma=1)
    return global_u, iter, message_pass_num, converge


In [8]:
def append_record(record, filename, typ):
    with open(filename, 'a') as f:
        if typ != '':
            json.dump(record, f, default = typ)
        else:
            json.dump(record, f)
        f.write('\n')
        # f.write(os.linesep)
        f.close()

In [10]:
run_num = 88
probability = 0.7
time_bound = 300
max_t_num = 1000
capNum = 10
max_capNum_task = 10
max_capNum_agent = 10
max_capVal = 10
capabilities = list(range(0, capNum))
a_min_edge = 1
min_t_num = 100
ex_identifier = 120

# t_max_edge = 6
# min_t_num = 2
# max_t_num = 2
# agent_num = 4
# agents = np.array([[0, 0, 0, 7, 6], [4, 3, 8, 8, 4], [0, 9, 8, 0, 0], [4, 9, 0, 7, 0]])
# tasks = np.array([[0, 1, 2], [0, 3, 4]])
# constraints = ([[0, 1], [0, 1], [0, 1], [0, 1]], [[0, 1, 2, 3], [0, 1, 2, 3]])

# compare algorithms
for run in range(0, run_num):
    print('run:', run)
    task_num = min_t_num
    while task_num <= max_t_num:
        t_max_edge = 0.04 * task_num
        ex_identifier += 1
        agent_num = 2 * task_num
        tasks = gen_tasks(task_num, max_capNum_task, capabilities)

        constraints = gen_constraints(agent_num, task_num, 1, a_min_edge, t_max_edge)
        ## caps_lists: capabilities the agent can provide; agents: the value of the agent provide the capabilities
        caps_lists, agents = gen_agents(constraints, tasks, max_capNum_agent, capabilities, max_capVal)

        a_tasks = constraints[0]
        t_agents = constraints[1]
        for i in range(0, agent_num):
            a_tasks[i].append(task_num)
        t_agents.append(list(range(0, agent_num)))

        result = {"ex_identifier": ex_identifier, "task_num": task_num, "agent_num": agent_num}

        if t_max_edge <= 17:
            start = time.time()
            new_con = OPD(agents, tasks, constraints, gamma=1)
            r = FMS(agents, tasks, new_con, gamma=1, time_bound=time_bound)
            end = time.time()
            result['BnBFMS_u'] = r[1]
            result['BnBFMS_t'] = end - start
            result['BnBFMS_iter'] = r[2]
            result['BnBFMS_converge'] = r[4]
            print("task_number: ", task_num, "BnBFMS time:", result['BnBFMS_t'], 'result:', result['BnBFMS_u'],
                  "iteration:", result['BnBFMS_iter'], "converge?", result['BnBFMS_converge'])
            print()
        
        r = random(agents, tasks,constraints, gamma=1 )
        alloc = r[0]
        result['rand'] = r[1]
        print("task_number: ", task_num, 'rand result',result['rand'])
        rand_CS = [[] for i in list(range(0, task_num + 1))]
        for i in list(range(0, agent_num)):
            rand_CS[alloc[i]].append(i)


         ##  DSA
        start = time.time()
        dsa_u, dsa_iter, dsa_msg_num, if_converge = embed_DSA(a_tasks, t_agents, tasks, agents, probability, time_bound)
        end = time.time()
        result['dsa_u'] = dsa_u
        result['dsa_t'] = end-start
        result['dsa_iter'] = dsa_iter
        result['dsa_msg'] = dsa_msg_num
        result['DSA_converge'] = if_converge
        print("task_number: ", task_num, "DSA time:", result['dsa_t'], 'result:', result['dsa_u'],
              "iteration:", result['dsa_iter'], "messages", result['dsa_msg'], "converge?", result['DSA_converge'])
        print()
        
        ##  DisNE starts from a random solution
        rand_disne = copy.deepcopy(rand_CS) 
        start = time.time()
        [LS_u, LS_iter, LS_msg_num, if_converge] = embed_disNE(a_tasks, t_agents, tasks, agents, time_bound, rand_disne)
        end = time.time()
        result['LS_u'] = LS_u
        result['LS_t'] = end - start
        result['LS_iter'] = LS_iter
        result['LS_msg_num'] = LS_msg_num
        result['LS_converge'] = if_converge
        print("task number: ", task_num, "LS_t:", result['LS_t'], 'LS_u:', LS_u,
                "LS_iter:", LS_iter, "messages", result['LS_msg_num'], "converge?", result['LS_converge'])
        print()

        start = time.time()
        disNE_u, disNE_iter, disNE_msg_num, if_converge = embed_disNE(a_tasks, t_agents, tasks, agents, time_bound, [])
        end = time.time()
        result['disNE_u'] = disNE_u
        result['disNE_t'] = end-start
        result['disNE_iter'] = disNE_iter
        result['DisNE_msg'] = disNE_msg_num
        result['DisNE_converge'] = if_converge
        print("task_number: ", task_num, "disNE time:", result['disNE_t'],'result:', result['disNE_u'],
              "iteration:", result['disNE_iter'], "messages", result['DisNE_msg'], "converge?", result['DisNE_converge'])
        print()

#         append data and result
        files = {'general_cap_thesis': [result, '']} #
        for filename in list(files.keys()):
            append_record(files[filename][0], filename, typ=files[filename][1])

        # increase the task_number
        task_num += 100
    run += 1




run: 0
task_number:  100 BnBFMS time: 3.3480045795440674 result: 3363 iteration: 300 converge? True

task_number:  100 rand result 1574
task_number:  100 DSA time: 0.03590726852416992 result: 3420 iteration: 11 messages 1332 converge? True

task number:  100 LS_t: 0.02987980842590332 LS_u: 3409 LS_iter: 7 messages 1991 converge? True

task_number:  100 disNE time: 0.03191518783569336 result: 3408 iteration: 7 messages 2096 converge? True

task_number:  200 BnBFMS time: 300.500417470932 result: 7767 iteration: 313 converge? True

task_number:  200 rand result 3774
task_number:  200 DSA time: 0.3520638942718506 result: 7800 iteration: 35 messages 13568 converge? True

task number:  200 LS_t: 0.11768770217895508 LS_u: 7660 LS_iter: 10 messages 8004 converge? True

task_number:  200 disNE time: 0.12170076370239258 result: 7745 iteration: 8 messages 7542 converge? True

task_number:  300 BnBFMS time: 316.16449213027954 result: 11357 iteration: 7 converge? False

task_number:  300 rand resul

task_number:  200 BnBFMS time: 300.75534558296204 result: 7364 iteration: 309 converge? False

task_number:  200 rand result 3602
task_number:  200 DSA time: 0.24334955215454102 result: 7475 iteration: 22 messages 9352 converge? True

task number:  200 LS_t: 0.14461374282836914 LS_u: 7370 LS_iter: 12 messages 8248 converge? True

task_number:  200 disNE time: 0.1326451301574707 result: 7408 iteration: 9 messages 7668 converge? True

task_number:  300 BnBFMS time: 317.2539210319519 result: 11665 iteration: 6 converge? False

task_number:  300 rand result 6878
task_number:  300 DSA time: 1.5179753303527832 result: 12364 iteration: 61 messages 59748 converge? True

task number:  300 LS_t: 0.3241465091705322 LS_u: 12092 LS_iter: 16 messages 17848 converge? True

task_number:  300 disNE time: 0.27128076553344727 result: 12161 iteration: 9 messages 15596 converge? True

task_number:  400 BnBFMS time: 788.8454542160034 result: 12514 iteration: 1 converge? False

task_number:  400 rand result 

task_number:  300 BnBFMS time: 316.78932785987854 result: 11009 iteration: 7 converge? False

task_number:  300 rand result 6288
task_number:  300 DSA time: 1.5406057834625244 result: 11696 iteration: 68 messages 61404 converge? True

task number:  300 LS_t: 0.2872323989868164 LS_u: 11448 LS_iter: 11 messages 17858 converge? True

task_number:  300 disNE time: 0.2672863006591797 result: 11578 iteration: 8 messages 15730 converge? True

task_number:  400 BnBFMS time: 714.2716417312622 result: 12041 iteration: 1 converge? False

task_number:  400 rand result 8854
task_number:  400 DSA time: 300.11830163002014 result: 13639 iteration: 3713 messages 12517936 converge? False

task number:  400 LS_t: 0.5373110771179199 LS_u: 15910 LS_iter: 14 messages 32602 converge? True

task_number:  400 disNE time: 0.500662088394165 result: 16105 iteration: 9 messages 26978 converge? True

task_number:  500 rand result 11378
task_number:  500 DSA time: 300.15093183517456 result: 16808 iteration: 2242 mes

task_number:  400 BnBFMS time: 742.8774607181549 result: 12070 iteration: 1 converge? False

task_number:  400 rand result 8737
task_number:  400 DSA time: 300.1330831050873 result: 13944 iteration: 3500 messages 11152256 converge? False

task number:  400 LS_t: 0.801044225692749 LS_u: 15680 LS_iter: 14 messages 31406 converge? True

task_number:  400 disNE time: 0.805161714553833 result: 15976 iteration: 10 messages 27203 converge? True

task_number:  500 rand result 11938
task_number:  500 DSA time: 300.1520872116089 result: 17028 iteration: 2185 messages 12395920 converge? False

task number:  500 LS_t: 0.807847261428833 LS_u: 20666 LS_iter: 15 messages 47492 converge? True

task_number:  500 disNE time: 0.7789220809936523 result: 20924 iteration: 12 messages 41821 converge? True

task_number:  600 rand result 15666
task_number:  600 DSA time: 300.2579083442688 result: 19735 iteration: 1400 messages 11969952 converge? False

task number:  600 LS_t: 1.2935559749603271 LS_u: 25760 LS_

task_number:  500 rand result 12082
task_number:  500 DSA time: 300.15397548675537 result: 17254 iteration: 2188 messages 12260560 converge? False

task number:  500 LS_t: 0.7689554691314697 LS_u: 20334 LS_iter: 14 messages 45472 converge? True

task_number:  500 disNE time: 0.7589871883392334 result: 20734 iteration: 11 messages 40901 converge? True

task_number:  600 rand result 15684
task_number:  600 DSA time: 300.1614291667938 result: 20352 iteration: 1432 messages 12517185 converge? False

task number:  600 LS_t: 1.1629106998443604 LS_u: 25643 LS_iter: 15 messages 67271 converge? True

task_number:  600 disNE time: 1.128981351852417 result: 26008 iteration: 11 messages 60462 converge? True

task_number:  700 rand result 18635
task_number:  700 DSA time: 300.2691812515259 result: 22656 iteration: 1039 messages 11838036 converge? False

task number:  700 LS_t: 1.9448268413543701 LS_u: 30308 LS_iter: 20 messages 95271 converge? True

task_number:  700 disNE time: 1.636641025543213 r

...

task number:  600 LS_t: 1.2616689205169678 LS_u: 25029 LS_iter: 19 messages 68288 converge? True

task_number:  600 disNE time: 1.1649000644683838 result: 25227 iteration: 12 messages 58574 converge? True

task_number:  700 rand result 17871
task_number:  700 DSA time: 300.22945523262024 result: 22452 iteration: 1061 messages 12043500 converge? False

task number:  700 LS_t: 1.7020668983459473 LS_u: 29841 LS_iter: 16 messages 93919 converge? True

task_number:  700 disNE time: 1.607964038848877 result: 30292 iteration: 13 messages 79205 converge? True

task_number:  800 rand result 22529
task_number:  800 DSA time: 300.5400195121765 result: 25823 iteration: 799 messages 11606656 converge? False

task number:  800 LS_t: 2.0469472408294678 LS_u: 35072 LS_iter: 15 messages 112572 converge? True

task_number:  800 disNE time: 2.0726563930511475 result: 35807 iteration: 11 messages 100823 converge? True

task_number:  900 rand result 24957
task_number:  900 DSA time: 300.54553961753845 resu

task_number:  700 disNE time: 1.6134326457977295 result: 30973 iteration: 12 messages 80825 converge? True

task_number:  800 rand result 22096
task_number:  800 DSA time: 300.5800018310547 result: 24966 iteration: 778 messages 11810346 converge? False

task number:  800 LS_t: 2.29794979095459 LS_u: 35214 LS_iter: 18 messages 123568 converge? True

task_number:  800 disNE time: 2.0665886402130127 result: 35827 iteration: 11 messages 103007 converge? True

task_number:  900 rand result 24486
task_number:  900 DSA time: 300.34757590293884 result: 28052 iteration: 619 messages 11424541 converge? False

task number:  900 LS_t: 2.8434371948242188 LS_u: 39030 LS_iter: 19 messages 148093 converge? True

task_number:  900 disNE time: 2.7483158111572266 result: 39836 iteration: 14 messages 127064 converge? True

task_number:  1000 rand result 27357
task_number:  1000 DSA time: 300.70529866218567 result: 31419 iteration: 505 messages 10832640 converge? False

task number:  1000 LS_t: 3.653763294

task_number:  900 rand result 25061
task_number:  900 DSA time: 300.73874855041504 result: 28566 iteration: 630 messages 11357748 converge? False

task number:  900 LS_t: 2.9840922355651855 LS_u: 39930 LS_iter: 22 messages 147162 converge? True

task_number:  900 disNE time: 2.8035128116607666 result: 40567 iteration: 13 messages 128460 converge? True

task_number:  1000 rand result 28587
task_number:  1000 DSA time: 300.5347213745117 result: 31270 iteration: 508 messages 10990600 converge? False

task number:  1000 LS_t: 3.7019402980804443 LS_u: 44195 LS_iter: 20 messages 184553 converge? True

task_number:  1000 disNE time: 3.4299144744873047 result: 44975 iteration: 13 messages 158113 converge? True

run: 83
task_number:  100 BnBFMS time: 3.0997486114501953 result: 3376 iteration: 300 converge? True

task_number:  100 rand result 1583
task_number:  100 DSA time: 0.039891958236694336 result: 3385 iteration: 14 messages 1392 converge? True

task number:  100 LS_t: 0.03091716766357422 

task number:  1000 LS_t: 3.5667457580566406 LS_u: 43908 LS_iter: 19 messages 177991 converge? True

task_number:  1000 disNE time: 3.476837396621704 result: 44707 iteration: 14 messages 159094 converge? True

run: 85
task_number:  100 BnBFMS time: 3.150834560394287 result: 3493 iteration: 300 converge? True

task_number:  100 rand result 1923
task_number:  100 DSA time: 0.02892303466796875 result: 3549 iteration: 8 messages 1160 converge? True

task number:  100 LS_t: 0.028946399688720703 LS_u: 3516 LS_iter: 7 messages 1876 converge? True

task_number:  100 disNE time: 0.03188943862915039 result: 3534 iteration: 7 messages 2155 converge? True

task_number:  200 BnBFMS time: 300.4802439212799 result: 7353 iteration: 388 converge? True

task_number:  200 rand result 3430
task_number:  200 DSA time: 0.22044086456298828 result: 7401 iteration: 19 messages 9176 converge? True

task number:  200 LS_t: 0.11965131759643555 LS_u: 7339 LS_iter: 9 messages 8376 converge? True

task_number:  200 d

task_number:  200 BnBFMS time: 300.48878145217896 result: 7374 iteration: 384 converge? False

task_number:  200 rand result 3758
task_number:  200 DSA time: 0.26828694343566895 result: 7380 iteration: 24 messages 9928 converge? True

task number:  200 LS_t: 0.12865829467773438 LS_u: 7273 LS_iter: 11 messages 8116 converge? True

task_number:  200 disNE time: 0.12466788291931152 result: 7373 iteration: 9 messages 7605 converge? True

task_number:  300 BnBFMS time: 315.0669503211975 result: 11319 iteration: 8 converge? False

task_number:  300 rand result 6087
task_number:  300 DSA time: 2.4415152072906494 result: 11822 iteration: 104 messages 97068 converge? True

task number:  300 LS_t: 0.28729796409606934 LS_u: 11590 LS_iter: 12 messages 18366 converge? True

task_number:  300 disNE time: 0.2752683162689209 result: 11675 iteration: 8 messages 16090 converge? True

task_number:  400 BnBFMS time: 695.9835608005524 result: 11780 iteration: 1 converge? False

task_number:  400 rand resul